1. datagen.py
Клітинка 1: Імпорти та завантаження моделей
python

In [3]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from transformers import pipeline
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

Імпортуємо необхідні бібліотеки, включаючи spaCy для роботи з NER.

Клітинка 2: Функція для створення CoNLL-формату

In [4]:
def generate_conll_format(texts, entities):
    dataset = []
    for text in texts:
        tokens = text.split()
        labels = ["O"] * len(tokens)
        for entity in entities:
            for i, token in enumerate(tokens):
                if token.startswith(entity):
                    labels[i] = "B-MOUNTAIN" if labels[i] == "O" else "I-MOUNTAIN"
        dataset.extend(zip(tokens, labels))
        dataset.append(("", ""))
    return dataset

Функція створює розмічені дані в форматі CoNLL із текстів та заданих сутностей.

Клітинка 3: Створення даних про гори

In [5]:
mountains = ["Everest", "Kilimanjaro", "Mont-Blanc", "Aconcagua", "Lhotse", "Nuptse"]

texts = [
    f"The mount {mountains[0]}, standing at an impressive 8,848 meters, {mountains[0]} is the highest mountain on Earth and a beacon for adventurers worldwide."
    for mountain in mountains
]

dataset = generate_conll_format(texts, mountains)


Створюємо список текстів із назвами гір і генеруємо розмітку.

Клітинка 4: Збереження датасету

In [6]:
with open("ner_dataset.conll", "w") as file:
    for token, label in dataset:
        file.write(f"{token} {label}\n")


Зберігаємо датасет у форматі CoNLL у файл.


2. mount.py


Клітинка 1:  Імпорти та базові функції

In [7]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

Імпортуємо бібліотеки для роботи з BERT і токенізацією.

Клітинка 2: Функція для парсингу CoNLL-файлів

In [8]:
def parse_conll(file_path):
    sentences, labels = [], []
    current_sentence, current_labels = [], []
    with open(file_path, "r", encoding="utf-8") as file:
        for line in file:
            line = line.strip()
            if line == "":
                if current_sentence:
                    sentences.append(current_sentence)
                    labels.append(current_labels)
                    current_sentence, current_labels = [], []
            else:
                token, tag = line.split()
                current_sentence.append(token)
                current_labels.append(tag)
    if current_sentence:
        sentences.append(current_sentence)
        labels.append(current_labels)
    return sentences, labels


Зчитуємо CoNLL-дані та зберігаємо токени та відповідні їм теги.

Клітинка 3: Токенізація та вирівнювання міток

In [9]:
def tokenize_and_align_labels(sentences, labels):
    tokenized_inputs = tokenizer(
        sentences,
        is_split_into_words=True,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    aligned_labels = []
    for i, label in enumerate(labels):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(tag2id[label[word_idx]])
            else:
                label_ids.append(tag2id[label[word_idx]])
            previous_word_idx = word_idx
        aligned_labels.append(label_ids)
    tokenized_inputs["labels"] = aligned_labels
    return tokenized_inputs


Використовуємо токенайзер для вирівнювання токенів та міток.

Клітинка 4: Підготовка датасету та токенайзера

In [10]:
file_path = "ner_dataset.conll"
sentences, labels = parse_conll(file_path)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-cased")

tag2id = {"O": 0, "B-MOUNTAIN": 1}
id2tag = {v: k for k, v in tag2id.items()}

tokenized_data = tokenize_and_align_labels(sentences, labels)


Завантажуємо датасет, налаштовуємо токенайзер і створюємо мітки.

Клітинка 5: Створення датасету для PyTorch

In [11]:
class NERDataset(Dataset):
    def __init__(self, tokenized_data):
        self.data = tokenized_data

    def __len__(self):
        return len(self.data["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.data.items()}

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    sentences, labels, test_size=0.2, random_state=42
)

train_dataset = NERDataset(tokenize_and_align_labels(train_sentences, train_labels))
val_dataset = NERDataset(tokenize_and_align_labels(val_sentences, val_labels))


Розбиваємо дані на тренувальний і валідаційний датасети.

Клітинка 6: Налаштування тренування

In [12]:
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(tag2id))

training_args = TrainingArguments(
    output_dir="./bert-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Misha\PycharmProjects\mountains\.venv\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Misha\AppData\Local\Temp\ipykernel_10096\2908876682.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Налаштовуємо модель і параметри тренування.

Клітинка 7: Тренування моделі

In [13]:
trainer.train()
trainer.save_model("./bert-ner")


C:\Users\Misha\AppData\Local\Temp\ipykernel_10096\152637357.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.data.items()}


Epoch,Training Loss,Validation Loss
1,No log,0.427011
2,No log,0.318423
3,No log,0.250810
4,No log,0.197148
5,No log,0.146047
6,No log,0.102733
7,No log,0.070225
8,No log,0.047477
9,No log,0.031124
10,No log,0.019781


C:\Users\Misha\AppData\Local\Temp\ipykernel_10096\152637357.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.data.items()}
C:\Users\Misha\AppData\Local\Temp\ipykernel_10096\152637357.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.data.items()}
C:\Users\Misha\AppData\Local\Temp\ipykernel_10096\152637357.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.data.item

Тренуємо модель і зберігаємо її.

Клітинка 8: Тестування та візуалізація

In [14]:
import spacy
from spacy.tokens import Doc, Span
from spacy import displacy

ner_model = pipeline("ner", model="./bert-ner", tokenizer="./bert-ner", aggregation_strategy="simple")

text = "Mount Everest is the biggest mountain in the world."

results = ner_model(text)

# Підготовка даних для spaCy
def convert_to_spacy_format(text, results):
    entities = []
    for entity in results:
        start = entity["start"]
        end = entity["end"]
        label = entity["entity_group"]
        entities.append({"start": start, "end": end, "label": label})
    return {"text": text, "ents": entities, "title": "Named Entities"}

# Конвертуємо результати
spacy_data = convert_to_spacy_format(text, results)

# Візуалізація за допомогою spaCy displacy
displacy.render(spacy_data, style="ent", manual=True, jupyter=True)

Використовуємо модель для розпізнавання сутностей і візуалізуємо їх.

